In [1]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 38.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [2]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu


In [3]:
pip install transformers datasets seqeval scikit-learn pandas numpy jupyter

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 92.3 MB/s  0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16250 sha256=3f50c3eef18324b510443250216a68536214bfdde934116b63bea1e452dece8b
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [jupyter]


In [5]:
import torch
print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

Device: cuda


In [6]:
from google.colab import files
uploaded = files.upload()

Saving ner.csv to ner.csv


In [7]:
import pandas as pd

df = pd.read_csv("ner.csv")

print("Shape:", df.shape)

df.head()

Shape: (47959, 4)


,Sentence #,Sentence,POS,Tag
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [8]:
import ast

print(df.columns.tolist())

def safe_literal_eval(x):
    if isinstance(x, list):
        return x
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return x

df["Sentence"] = df["Sentence"].astype(str)
df["POS"] = df["POS"].apply(safe_literal_eval)
df["Tag"] = df["Tag"].apply(safe_literal_eval)

sentences = df["Sentence"].tolist()
pos_tags = df["POS"].tolist()
ner_tags = df["Tag"].tolist()

print("Example sentence:", sentences[0])
print("POS tags len:", len(pos_tags[0]))
print("NER tags len:", len(ner_tags[0]))
print("POS tags:", pos_tags[0][:10])
print("NER tags:", ner_tags[0][:10])


['Sentence #', 'Sentence', 'POS', 'Tag']
Example sentence: Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
POS tags len: 24
NER tags len: 24
POS tags: ['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP', 'TO', 'VB', 'DT']
NER tags: ['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O']


In [9]:
unique_tags = sorted({tag for sent_tags in ner_tags for tag in sent_tags})
print("Unique tags:", unique_tags)
print("Number of tags:", len(unique_tags))

label2id = {label: idx for idx, label in enumerate(unique_tags)}
id2label = {idx: label for label, idx in label2id.items()}

print("label2id:", label2id)

Unique tags: ['B-art', 'B-eve', 'B-geo', 'B-gpe', 'B-nat', 'B-org', 'B-per', 'B-tim', 'I-art', 'I-eve', 'I-geo', 'I-gpe', 'I-nat', 'I-org', 'I-per', 'I-tim', 'O']
Number of tags: 17
label2id: {'B-art': 0, 'B-eve': 1, 'B-geo': 2, 'B-gpe': 3, 'B-nat': 4, 'B-org': 5, 'B-per': 6, 'B-tim': 7, 'I-art': 8, 'I-eve': 9, 'I-geo': 10, 'I-gpe': 11, 'I-nat': 12, 'I-org': 13, 'I-per': 14, 'I-tim': 15, 'O': 16}


In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "bert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

num_labels = len(unique_tags)

model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

print("Tokenizer and model loaded.")
print("Number of labels in model:", model.config.num_labels)
print("Some config labels:", model.config.id2label)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizer and model loaded.
Number of labels in model: 17
Some config labels: {0: 'B-art', 1: 'B-eve', 2: 'B-geo', 3: 'B-gpe', 4: 'B-nat', 5: 'B-org', 6: 'B-per', 7: 'B-tim', 8: 'I-art', 9: 'I-eve', 10: 'I-geo', 11: 'I-gpe', 12: 'I-nat', 13: 'I-org', 14: 'I-per', 15: 'I-tim', 16: 'O'}


In [11]:
from typing import List

max_length = 128

def tokenize_and_align_labels(sentences: List[str], ner_tags: List[List[str]]):
    input_ids_list = []
    attention_masks_list = []
    label_ids_list = []

    for sent, tags in zip(sentences, ner_tags):
        words = sent.split()

        if len(words) != len(tags):
            print("Length mismatch, skipping sentence.")
            print("Sentence:", sent)
            print("len(words):", len(words), "len(tags):", len(tags))
            continue

        encoding = tokenizer(
            words,
            is_split_into_words=True,
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_tensors="pt"
        )

        input_ids = encoding["input_ids"][0]
        attention_mask = encoding["attention_mask"][0]

        word_ids = encoding.word_ids(batch_index=0)

        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                if word_idx != previous_word_idx:
                    label_ids.append(label2id[tags[word_idx]])
                else:
                    label_ids.append(-100)
            previous_word_idx = word_idx

        input_ids_list.append(input_ids)
        attention_masks_list.append(attention_mask)
        label_ids_list.append(torch.tensor(label_ids))

    input_ids_tensor = torch.stack(input_ids_list)
    attention_masks_tensor = torch.stack(attention_masks_list)
    label_ids_tensor = torch.stack(label_ids_list)

    return input_ids_tensor, attention_masks_tensor, label_ids_tensor

small_sentences = sentences[:10]
small_ner_tags = ner_tags[:10]

input_ids_small, attention_masks_small, labels_small = tokenize_and_align_labels(
    small_sentences,
    small_ner_tags
)

print("Shapes:")
print("input_ids_small:", input_ids_small.shape)
print("attention_masks_small:", attention_masks_small.shape)
print("labels_small:", labels_small.shape)

idx = 0
tokens = tokenizer.convert_ids_to_tokens(input_ids_small[idx])
print("Tokens:", tokens[:30])
print("Labels:", labels_small[idx][:30])


Shapes:
input_ids_small: torch.Size([10, 128])
attention_masks_small: torch.Size([10, 128])
labels_small: torch.Size([10, 128])
Tokens: ['[CLS]', 'Thousands', 'of', 'demons', '##tra', '##tors', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.', '[SEP]', '[PAD]', '[PAD]']
Labels: tensor([-100,   16,   16,   16, -100, -100,   16,   16,   16,    2,   16,   16,
          16,   16,   16,    2,   16,   16,   16,   16,   16,    3,   16,   16,
          16,   16,   16, -100, -100, -100])


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    sentences,
    ner_tags,
    test_size=0.1,
    random_state=42
)

print("Train size:", len(X_train))
print("Test size:", len(X_test))

train_input_ids, train_attention_masks, train_labels = tokenize_and_align_labels(X_train, y_train)
test_input_ids, test_attention_masks, test_labels = tokenize_and_align_labels(X_test, y_test)

print("Train tensors:")
print(" input_ids:", train_input_ids.shape)
print(" attention_masks:", train_attention_masks.shape)
print(" labels:", train_labels.shape)

print("Test tensors:")
print(" input_ids:", test_input_ids.shape)
print(" attention_masks:", test_attention_masks.shape)
print(" labels:", test_labels.shape)


Train size: 43163
Test size: 4796
Length mismatch, skipping sentence.
Sentence: " And I think they 'll want a one-stop shop in terms of combining security , immigration , customs , and quarantine together  just to make sure it 's more streamlined and provides more certainty . "
len(words): 37 len(tags): 38
Length mismatch, skipping sentence.
Sentence: He says they and about 300 party supporters are trying to force their way through police barricades  but so far have been unsuccessful at doing so . "
len(words): 28 len(tags): 29
Length mismatch, skipping sentence.
Sentence: U.S. weather forecasters say Hurricane Wilma has strengthened to a powerful category 5 storm and a key low-pressure measurement indicates it is the most powerful storm of the year .
len(words): 30 len(tags): 29
Length mismatch, skipping sentence.
Sentence: In a telephone interview to discuss the issues at stake , he told VOA reporter Akwei Thompson the opposition would demonstrate a stronger political will to tackl

In [18]:
from torch.utils.data import Dataset, DataLoader

class NERDataset(Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return self.input_ids.shape[0]

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_masks[idx],
            "labels": self.labels[idx],
        }

train_dataset = NERDataset(train_input_ids, train_attention_masks, train_labels)
test_dataset = NERDataset(test_input_ids, test_attention_masks, test_labels)

batch_size = 16

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

batch = next(iter(train_dataloader))
print("Batch input_ids shape:", batch["input_ids"].shape)
print("Batch labels shape:", batch["labels"].shape)


Batch input_ids shape: torch.Size([16, 128])
Batch labels shape: torch.Size([16, 128])


In [19]:
import torch
from torch.optim import AdamW
from tqdm.auto import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

epochs = 2

for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    model.train()
    total_loss = 0.0

    progress_bar = tqdm(train_dataloader)

    for batch in progress_bar:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({"loss": loss.item()})

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss:.4f}")


Using device: cuda

Epoch 1/2


  0%|          | 0/2698 [00:00<?, ?it/s]

Average training loss: 0.1097

Epoch 2/2


  0%|          | 0/2698 [00:00<?, ?it/s]

Average training loss: 0.0776


In [20]:
import numpy as np
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score  # [web:74]

model.eval()

all_true_labels = []
all_pred_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        predictions = torch.argmax(logits, dim=-1)

        predictions = predictions.cpu().numpy()
        true_labels = labels.cpu().numpy()

        for pred_seq, true_seq in zip(predictions, true_labels):
            pred_tags = []
            true_tags = []

            for p, t in zip(pred_seq, true_seq):
                if t == -100:
                    continue

                true_tags.append(id2label[int(t)])
                pred_tags.append(id2label[int(p)])

            all_true_labels.append(true_tags)
            all_pred_labels.append(pred_tags)

precision = precision_score(all_true_labels, all_pred_labels)
recall = recall_score(all_true_labels, all_pred_labels)
f1 = f1_score(all_true_labels, all_pred_labels)

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")

print("\nClassification report:\n")
print(classification_report(all_true_labels, all_pred_labels, digits=4))


Precision: 0.8378
Recall:    0.8380
F1-score:  0.8379

Classification report:

              precision    recall  f1-score   support

         art     0.3077    0.0976    0.1481        41
         eve     0.6000    0.2308    0.3333        26
         geo     0.8515    0.9117    0.8806      3894
         gpe     0.9517    0.9449    0.9483      1542
         nat     0.4286    0.3214    0.3673        28
         org     0.7269    0.6816    0.7035      1988
         per     0.7798    0.7665    0.7731      1696
         tim     0.8832    0.8580    0.8704      2035

   micro avg     0.8378    0.8380    0.8379     11250
   macro avg     0.6912    0.6015    0.6281     11250
weighted avg     0.8345    0.8380    0.8353     11250



In [21]:
import os

save_dir = "ner_bert_model"
os.makedirs(save_dir, exist_ok=True)

model_to_save = model.module if hasattr(model, "module") else model
model_to_save.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print("Saved to:", os.listdir(save_dir))


Saved to: ['special_tokens_map.json', 'tokenizer_config.json', 'config.json', 'vocab.txt', 'model.safetensors', 'tokenizer.json']


In [22]:
from google.colab import files
!zip -r ner_bert_model.zip ner_bert_model
files.download("ner_bert_model.zip")

  adding: ner_bert_model/ (stored 0%)
  adding: ner_bert_model/special_tokens_map.json (deflated 42%)
  adding: ner_bert_model/tokenizer_config.json (deflated 75%)
  adding: ner_bert_model/config.json (deflated 57%)
  adding: ner_bert_model/vocab.txt (deflated 49%)
  adding: ner_bert_model/model.safetensors (deflated 7%)
  adding: ner_bert_model/tokenizer.json (deflated 70%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

inference_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

loaded_tokenizer = AutoTokenizer.from_pretrained("ner_bert_model")
loaded_model = AutoModelForTokenClassification.from_pretrained("ner_bert_model")
loaded_model.to(inference_device)
loaded_model.eval()

print("Loaded model for inference.")


Loaded model for inference.


In [24]:
def ner_predict(text):
    encoding = loaded_tokenizer(
        text.split(),
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True,
        max_length=128,
        padding="max_length"
    )

    input_ids = encoding["input_ids"].to(inference_device)
    attention_mask = encoding["attention_mask"].to(inference_device)

    with torch.no_grad():
        outputs = loaded_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)[0].cpu().numpy()

    word_ids = encoding.word_ids(batch_index=0)

    words = text.split()
    entities = []

    current_entity = None
    current_label = None

    for idx, word_idx in enumerate(word_ids):
        if word_idx is None:
            continue

        word = words[word_idx]
        label_id = int(predictions[idx])
        label = id2label[label_id]

        if label == "O":
            if current_entity is not None:
                entities.append({"text": " ".join(current_entity), "label": current_label})
                current_entity = None
                current_label = None
            continue

        tag, ent_type = label.split("-")
        if tag == "B":
            if current_entity is not None:
                entities.append({"text": " ".join(current_entity), "label": current_label})
            current_entity = [word]
            current_label = ent_type
        elif tag == "I":
            if current_entity is not None and current_label == ent_type:
                current_entity.append(word)
            else:
                current_entity = [word]
                current_label = ent_type

    if current_entity is not None:
        entities.append({"text": " ".join(current_entity), "label": current_label})

    return entities

test_sentence = "Barack Obama visited London in 2010 ."
print(test_sentence)
print(ner_predict(test_sentence))


Barack Obama visited London in 2010 .
[{'text': 'Barack Obama', 'label': 'per'}, {'text': 'London', 'label': 'geo'}, {'text': '2010', 'label': 'tim'}]


In [25]:
!pip install gradio


In [26]:
import gradio as gr

def gradio_ner(text):
    entities = ner_predict(text)
    if not entities:
        return "No entities found."
    lines = [f"{e['text']} -> {e['label']}" for e in entities]
    return "\n".join(lines)

demo = gr.Interface(
    fn=gradio_ner,
    inputs=gr.Textbox(lines=3, label="Input text"),
    outputs=gr.Textbox(label="Detected entities"),
    title="BERT NER Demo",
    description="Fine-tuned BERT NER model (geo, gpe, org, per, tim, etc.)."
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3d78449908a570f0e4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
